In [1]:
#PART 0 - IMPORT MODULES

import sys
import glob
import os
import numpy as np
import xml.etree.ElementTree as ET
from lxml import etree

#import arcpy
import csv
import tempfile
import codecs
import cStringIO
import time
from xml.etree.ElementTree import ElementTree


In [2]:
#PART 1 - DECLARE INPUT FILES

#csvFileName = input("Enter input CSv file path preceded by the letter r (e.g., r\"C:\Users\...\"): ")
#xmlInputPath = input("Enter input XMLs directory path preceded by the letter r (e.g., r\"C:\Users\...\"): ")
#xmlOutputPath = input("Enter output XMLs directory path preceded by the letter r (e.g., r\"C:\Users\...\"): ")

#TEST WITH MGM SAMPLE FILES 
#csvFileName = r"C:\Temp_Jamaica\DN_GDB\MGM_local_GDB\Template_metadata_import_sheet_MGM.csv"
#xmlInputPath = r"C:\Temp_Jamaica\DN_GDB\MGM_local_GDB\Output_XML"
#xmlTempPath = r"C:\Temp_Jamaica\DN_GDB\MGM_local_GDB\Temp_XML"
#xmlOutputPath = r"C:\Temp_Jamaica\DN_GDB\MGM_local_GDB\Updated_XML"

#TEST WITH MGM SAMPLE FILES 
#csvFileName = r"C:\Users\marcg\Dropbox\GWP\Temp_Jobs\Jamaica NSP\metadata_import_process\1 Python_CSV_Input\Output to python from Combined MGM.csv"
#xmlInputPath = r"C:\Users\marcg\Dropbox\GWP\Temp_Jobs\Jamaica NSP\metadata_import_process\2 Orginal_xml_files"
#xmlTempPath = r"C:\Users\marcg\Dropbox\GWP\Temp_Jobs\Jamaica NSP\metadata_import_process\3 Temp_metadata_xml_file"
#xmlOutputPath = r"C:\Users\marcg\Dropbox\GWP\Temp_Jobs\Jamaica NSP\metadata_import_process\4 Metadata_updated_xml_file"

#J DRIVE FILES  
csvFileName = r"J:\David Jarvis\JAMNSP Jamaica NSP\ArcGIS\Metadata\metadata_import_process\1 Python_CSV_Input\Output to Python from Combined.csv"
xmlInputPath = r"J:\David Jarvis\JAMNSP Jamaica NSP\ArcGIS\Metadata\metadata_import_process\2 Original_xml_files"
xmlTempPath = r"J:\David Jarvis\JAMNSP Jamaica NSP\ArcGIS\Metadata\metadata_import_process\3 Temp_metadata_xml_file"
xmlOutputPath = r"J:\David Jarvis\JAMNSP Jamaica NSP\ArcGIS\Metadata\metadata_import_process\4 Metadata_updated_xml_file"
#xmlOutputPath = r"J:\David Jarvis\JAMNSP Jamaica NSP\ArcGIS\Metadata\metadata_import_process\MGM_TEMP_Metadata_updated_xml_file"

#xmlInput = r"C:\Users\marcg\Dropbox\GWP\Temp_Jobs\Jamaica NSP\Python_testing\from DN\ISO_template_feature.xml"
#xmlOutputPath = r"C:\Users\marcg\Dropbox\GWP\Temp_Jobs\Jamaica NSP\Python_testing\xml_output"

csvFileName.replace("\\","\\\\")
xmlInputPath.replace("\\","\\\\")
xmlTempPath.replace("\\","\\\\")
xmlOutputPath.replace("\\","\\\\")

'J:\\\\David Jarvis\\\\JAMNSP Jamaica NSP\\\\ArcGIS\\\\Metadata\\\\metadata_import_process\\\\4 Metadata_updated_xml_file'

In [3]:
#PART 2 - DEFINE DELIMITER FOR CSV FILE 

delimiter = "," # "\t" "|" # delimiter used in the CSV file(s)

In [4]:
#PART 3 - LOOP OVER CSV FILE LINES (TO GENERATE XML FILE NAMES) & RETRIEVE CSV HEADERS (XML ELEMENTS)

csvFile = open(csvFileName, 'rb')
csvFileRead = csv.reader(csvFile)
 
line_count = 0
csvHeaders = []
csvData = []
xmlFileNames = []

for line in csvFileRead:
    
    if line_count == 0:
        csvHeaders=line[1:]
    elif line_count == 1:
        pass
    else:
        #print len(line)
        xmlPathName = line[0]
        xmlPathName = xmlPathName + '.xml'
        xmlFileName = xmlPathName.split("\\")
        xmlFileName = xmlFileName[-1]                  
        #xmlFile = xmlFileName[0:-4] + '.xml'
        
        xmlFileNames.append(xmlFileName)
        csvData.append(line[1:])
        #print xmlFileName
        #print xmlPathName
        #print line[1:]
        #print "\n"
        
    line_count += 1
    
#for 
#tags = csvData.pop(0).strip().replace(', ','<;;;>').split(delimiter)
#for row in csvData:
    #rowencoded = row.encode('ascii')
    #rowData = rowencoded.replace(', ','<;;;>').split(delimiter)
    #print len(rowData)

In [5]:
# PART 4 - UPDATE XML FILES TO INCLUDE REQUIRED TREE STRUCTURE AND SAVE TEMPORARY XML FILES 

MissingFiles = []
line_count = 0

total_tic = time.clock()

for line in csvData:
    
    tic = time.clock()
    
    xmlInput = xmlInputPath + "\\" + str(xmlFileNames[line_count])
    #print xmlInput
    
    if os.path.exists(xmlInput):
    
        tree = etree.parse(xmlInput)
        root = tree.getroot()

        #root.tag
        #print "xmlInput:" + str(xmlInput)

        save_count = 0
        e_count = 0

        for h in csvHeaders: 

            h_element_split = h.split("/")
            #print h_element_split
            #h_element_name = h_element_split[-1]


            for i in range(0,len(h_element_split)):

                h_element_name = h_element_split[i]
                h_element_name_split = h_element_name.split("_")
                h_element_name = h_element_name_split[0]

                #print h_element_name

                if i == 0:
                    h_element_path = "/metadata"
                    #print "metadata = " + str(h_element_path)
                    if i == len(h_element_split)-1:
                        h_element_name = h_element_name#[:-4]
                    else:
                        pass                        
                elif i == len(h_element_split)-1:
                    #print i
                    h_element_path = "/".join(h_element_split[:i])
                    h_element_path = "/metadata/" + str(h_element_path)
                    h_element_name = h_element_name#[:-4]
                else:
                    h_element_path = "/".join(h_element_split[:i])
                    h_element_path = "/metadata/" + str(h_element_path)

                h_element_fullpath = str(h_element_path)+"/"+str(h_element_name)

                #print "HH = " + str(h_element_path)
                #print "EE = " + str(h_element_name)
                #print "FP = " + str(h_element_fullpath)
                #print i

                #len_h_element_name = len(h_element_name)
                #h_element_path = "/metadata/" + str(h[:-len_h_element_name])
                #h_element_name = h_element_name[:-4]
                #print h
                #print "Element = " + str(element_split)

                for e in root.iter():      
                    e_path = tree.getpath(e)
                    #e_elementpath = tree.getelementpath(e)
                    e_elementpath = tree.getpath(e)
                    #print "E =" + str(e_elementpath)
                    #print "H =" + str(h_element_path)
                    if str(e_elementpath) == h_element_fullpath:
                        #print "EXISTING\n"
                        break       
                else:
                    for e2 in root.iter():
                        e_elementpath = tree.getpath(e2)
                        #print "EE_1 = " + str(e_elementpath)
                        #print "HH_1 = " + str(h_element_path)
                        if str(e_elementpath) == h_element_path:
                            etree.SubElement(e2,str(h_element_name))
                            #print "FP =" + str(h_element_fullpath)
                            #print "UPDATED\n"
                            #print(etree.tostring(e, pretty_print=True))
                            #print "\n"
                            break
                    else:
                        print "ERROR\n" 

            #h_count +=1

        #for e3 in root.iter():
        #    e_path = tree.getpath(e3)
        #    print e_path

        tree = etree.ElementTree(root)                   

        outFilePath = str(xmlTempPath) + "\\" + str(xmlFileNames[line_count])    

        outFile = open(outFilePath, 'w')
        tree.write(outFile,encoding="UTF-8", xml_declaration=True, method="xml")
        #tree_tostring = etree.tostring(tree, xml_declaration=True)
        #tree_tostring.write(outFile)
        outFile.close()

        toc = time.clock()
        elapsed = toc-tic

        #print outFilePath  
        print "File No." + str(line_count+1) + " | Filename: " + str(xmlFileNames[line_count]) + " | Run time: " + str(elapsed)
        
    else:
        print "File missing: " + str(xmlInput)
        MissingFiles.append(xmlInput)
        
    line_count += 1
    
    
total_toc = time.clock()    
total_elapsed = total_toc - total_tic

print "\n" + "Number of XML files created: " + str(line_count) + " | Total run time: " + str(total_elapsed)  

File No.1 | Filename: AdaptationFund_Negril_Asset.xml | Run time: 0.0765213611856
File No.2 | Filename: admin_zones_NSWMA.xml | Run time: 0.109173319539
File No.3 | Filename: Administrative_Buildings_AnnottoBay.xml | Run time: 0.0807428462738
File No.4 | Filename: Administrative_Communities_SDC.xml | Run time: 0.113022830417
File No.5 | Filename: Administrative_Communities_STATIN.xml | Run time: 0.0988238148822
File No.6 | Filename: Administrative_Constituencies.xml | Run time: 0.0955764844749
File No.7 | Filename: Administrative_ED_2001.xml | Run time: 0.102892421681
File No.8 | Filename: Administrative_Parishes.xml | Run time: 0.0957480281367
File No.9 | Filename: administrative_regions_proposed.xml | Run time: 0.115578653213
File No.10 | Filename: Administrative_Settlements.xml | Run time: 0.116398596364
File No.11 | Filename: Agriculture_and_Residential_Transitional_zone.xml | Run time: 3.80823285141
File No.12 | Filename: agriculture_SantaCruz.xml | Run time: 0.0988140377823
File 

File No.111 | Filename: Commercial_LU_Zone.xml | Run time: 3.65748108131
File No.112 | Filename: commercial_site_Manchester_proposed.xml | Run time: 0.101981818149
File No.113 | Filename: community_centres.xml | Run time: 0.0989940253031
File No.114 | Filename: Community_Names.xml | Run time: 0.0827111542489
File No.115 | Filename: coneys_Hutia_Gbrownii.xml | Run time: 0.0800762258261
File No.116 | Filename: constituency_boundaries.xml | Run time: 0.119161515913
File No.117 | Filename: Coral_Monitoring_Sites07.xml | Run time: 0.0678006324895
File No.118 | Filename: Coral_reef_map_2008_RC_JA.xml | Run time: 0.0926464654006
File No.119 | Filename: coral_sites.xml | Run time: 0.0909545827045
File No.120 | Filename: court_houses.xml | Run time: 0.107618316241
File No.121 | Filename: craft_markets.xml | Run time: 0.099076241825
File No.122 | Filename: CriticalFacilities_Cemetaries.xml | Run time: 0.0828467004065
File No.123 | Filename: CriticalFacilities_Daycare.xml | Run time: 0.0913118912

File No.215 | Filename: Geology_Soils.xml | Run time: 0.111839356916
File No.216 | Filename: Geology_Stratadip.xml | Run time: 0.0925749148059
File No.217 | Filename: Geology_StrataDip_Point.xml | Run time: 0.0916469791428
File No.218 | Filename: Geology_SymbolGeoFormation.xml | Run time: 0.0972337029077
File No.219 | Filename: Goat_Island_Excavation_and_Reclaimation_Works.xml | Run time: 0.0557108040509
File No.220 | Filename: Green_Space_LU_Zone.xml | Run time: 3.65025269359
File No.221 | Filename: guano_caves.xml | Run time: 0.075804077584
File No.222 | Filename: Gullies_drains_rivamp.xml | Run time: 0.085175427837
File No.223 | Filename: habitat.xml | Run time: 0.0858002734033
File No.224 | Filename: HanoverExistingLanduse.xml | Run time: 0.107005469843
File No.225 | Filename: HanoverProposals.xml | Run time: 0.295377742699
File No.226 | Filename: harmons.xml | Run time: 0.0575191231186
File No.227 | Filename: HazVul_Earthquake_Buffer_SC.xml | Run time: 0.0863828996745
File No.228 

File No.318 | Filename: Jamaica_caves.xml | Run time: 0.0711292905914
File No.319 | Filename: jm_terr_target_caves_JAD2001.xml | Run time: 0.0575773413043
File No.320 | Filename: Kingston.xml | Run time: 0.0476331418799
File No.321 | Filename: Kingston_Excavation_and_Reclaimation_Works.xml | Run time: 0.0609735502783
File No.322 | Filename: Kingston_Habour_Wetlands.xml | Run time: 0.0599798413977
File No.323 | Filename: KSA_Proposals_2017_04_19.xml | Run time: 1.29584526584
File No.324 | Filename: Land_Parcels_Annotto_Bay.xml | Run time: 0.123269675525
File No.325 | Filename: land_use_Jamaica.xml | Run time: 0.0955644853068
File No.326 | Filename: land_use_Manchester_existing.xml | Run time: 0.0968608398707
File No.327 | Filename: land_use_Portland_proposed.xml | Run time: 0.0978207733153
File No.328 | Filename: Landslide.xml | Run time: 0.0763898147506
File No.329 | Filename: landslide_historical_events_1895to2009.xml | Run time: 0.0973341403885
File No.330 | Filename: landslide_vulne

File No.425 | Filename: pedestrian_crossings_NWA.xml | Run time: 0.10829604703
File No.426 | Filename: pharmacies.xml | Run time: 0.0911625682842
File No.427 | Filename: PIPELINE.xml | Run time: 0.0785705524417
File No.428 | Filename: pipelines_network_NIC.xml | Run time: 0.104984276646
File No.429 | Filename: pipelines_network_NWC.xml | Run time: 0.114532947938
File No.430 | Filename: police.xml | Run time: 0.0706604342099
File No.431 | Filename: pondbased.xml | Run time: 0.0592750013777
File No.432 | Filename: Port_Antonion.xml | Run time: 0.0477873534102
File No.433 | Filename: Port_Maria.xml | Run time: 0.0469860756321
File No.434 | Filename: Portland_Bight_Historic_Sites.xml | Run time: 0.0722656562478
File No.435 | Filename: Portland_Bight_protected_Area_Ports.xml | Run time: 0.082505835151
File No.436 | Filename: Portland_Bight_Ramsar_Wetland.xml | Run time: 0.103208399773
File No.437 | Filename: Portland_Blight_2007_Major_Land_Use.xml | Run time: 0.0726336307349
File No.438 | F

File No.534 | Filename: storm_surge_HAllen_1980.xml | Run time: 0.114012539575
File No.535 | Filename: storm_surge_Montego_Bay_100yr.xml | Run time: 0.0951405147021
File No.536 | Filename: strata.xml | Run time: 0.0930966564096
File No.537 | Filename: Stream_gauge_stations.xml | Run time: 0.125493965752
File No.538 | Filename: Sub_Watershed_management_units.xml | Run time: 0.0924438127846
File No.539 | Filename: subdivisions_StElizabeth.xml | Run time: 0.152777851847
File No.540 | Filename: SubRegional_Centre_Service_Area.xml | Run time: 0.0817738858995
File No.541 | Filename: substrates_ik.xml | Run time: 0.0576160052903
File No.542 | Filename: Surface_water_sources.xml | Run time: 0.141323979315
File No.543 | Filename: SW_Goat_Island_Container_Hub_Bentic_Coverage.xml | Run time: 0.0622703492558
File No.544 | Filename: swamp.xml | Run time: 0.0636635859914
File No.545 | Filename: swift_river_chelsea.xml | Run time: 0.057267585003
File No.546 | Filename: swift_river_hope_bay.xml | Run 

In [6]:
# PART 5 - NESTED LOOPS OVER: I) CSV FILE LINES, II) MASTER XML ELEMENT STRUCTURE & III) CSV HEADERS; TO FILL AND GENERATE XML FILES  

MissingFiles_v2 = []
line_count = 0

total_tic = time.clock()

for line in csvData:
        
    tic = time.clock()
    
    xmlInput = xmlTempPath + "\\" + str(xmlFileNames[line_count])
    #print xmlInput
    
    if os.path.exists(xmlInput):
    
        tree = etree.parse(xmlInput)
        root = tree.getroot()

        #root.tag
        #print "xmlInput:" + str(xmlInput)

        save_count = 0
        e_count = 0

        h_count=-1
        for h in csvHeaders: 
            h_count=h_count+1

            h_split = h.split("_")
            #print h_split

            for e in root.iter():      
                e_path = tree.getpath(e)
                #e_elementpath = tree.getelementpath(e)
                e_elementpath = tree.getpath(e)
                #print "e_path:" +str(e_path)
                #print "e_elementpath:" +str(e_elementpath)
                #print "\n"

                #e.text = ""
                #e.tail = ""
                #d = e.attrib
                #for key in d:
                    #d[key]=""

                #e_count = e_count + 1

                e_elementpath_len = len(str(e_elementpath)+"_")-10

                #print "EE =" + str(e_elementpath)
                #print "HH =" + "/metadata/"+str(h)#[:e_elementpath_len])



                #if str(e_elementpath)+"_" == "/metadata/" + str(h[:e_elementpath_len]):
                if str(e_elementpath) == "/metadata/" + str(h_split[0]):
                    #print e_elementpath
                    #print h_split
                    #print "FOUND"

                    #print h_split[1]

                    if h_split[1] == "tex":
                        #print "h: "+str(h)
                        #print "newtext:"+str(line[h_count])
                        e.text = line[h_count]
                        #print "oldtext:"+str(e.text)
                        if e.tag == "keyword2":
                            e.tag = "keyword"
                            #print e.tag
                        else:
                            pass
                    elif h_split[1] == "tag":
                        #e.tag = line[h_count]
                        #print "tag:"+str(e.tag)
                        pass
                    elif h_split[1] == "tai":
                        e.tail = line[h_count]
                        #print "tail:"+str(e.tail)
                    elif h_split[1] == "att":
                        #print "ATT"
                        d = e.attrib 
                        #print "att:"+str(d)
                        for key in d:
                            #if key == "value":
                                #print type(d[key])
                                #print d[key]
                                #pass
                            #else:
                                #pass
                            #print h_split[2]
                            if key == h_split[2]:
                                if key == "value":
                                    #print "Updated existing key: "+str(d)
                                    #print line[h_count]
                                    #numformat = float(line[h_count])
                                    d[key]="009"#'"'+str("%03d"%numformat)+'"'
                                    #d[key]=line[h_count]
                                    #print d[key]
                                    #print type(d[key])
                                else:
                                    d[key]=line[h_count]
                                    #print "Updated existing key: "+str(d)
                                #print e.attrib
                                break
                        else:
                            key = h_split[2]
                            if key == "value":
                                #print line[h_count]
                                #numformat = float(line[h_count])
                                d[key]="009"#'"'+str("%03d"%numformat)+'"'
                                #d[key]=line[h_count]
                                #print "Updated new key: "+str(d)
                                #print d[key]
                                #print type(d[key])
                            else:
                                d[key] = line[h_count]
                                #print "Updated new key: "+str(d)
                            #print d
                    else:
                        #print e_elementpath
                        #print h
                        print "ERROR1\n" # flag
                        pass
                    break  

            else:
                print "ERROR2\n"
                #print "Element: "+str(e_elementpath)
                print "CSV field:"+"/metadata/" + str(h) 
                pass

        outFilePath = str(xmlOutputPath) + "\\" + str(xmlFileNames[line_count])    

        outFile = open(outFilePath, 'w')
        tree.write(outFile,encoding="UTF-8", xml_declaration=True, method="xml")
        #tree.write(outFile,encoding="ascii", xml_declaration=True, method="xml")
        #tree_tostring = etree.tostring(tree, xml_declaration=True)
        #tree_tostring.write(outFile)
        outFile.close()

        toc = time.clock()
        elapsed = toc-tic

        #print outFilePath  
        print "File No." + str(line_count+1) + " | Filename: " + str(xmlFileNames[line_count]) + " | Run time: " + str(elapsed)
    
    else:
        MissingFiles_v2.append(xmlInput)
        
    line_count += 1
    
total_toc = time.clock()    
total_elapsed = total_toc - total_tic

print "\n" + "Number of XML files created: " + str(line_count) + " | Total run time: " + str(total_elapsed)  


File No.1 | Filename: AdaptationFund_Negril_Asset.xml | Run time: 0.0442689306875
File No.2 | Filename: admin_zones_NSWMA.xml | Run time: 0.055424601672
File No.3 | Filename: Administrative_Buildings_AnnottoBay.xml | Run time: 0.0449431061668
File No.4 | Filename: Administrative_Communities_SDC.xml | Run time: 0.0666620447649
File No.5 | Filename: Administrative_Communities_STATIN.xml | Run time: 0.0682148259943
File No.6 | Filename: Administrative_Constituencies.xml | Run time: 0.0599745084341
File No.7 | Filename: Administrative_ED_2001.xml | Run time: 0.0552477250466
File No.8 | Filename: Administrative_Parishes.xml | Run time: 0.0537318301486
File No.9 | Filename: administrative_regions_proposed.xml | Run time: 0.0615268452498
File No.10 | Filename: Administrative_Settlements.xml | Run time: 0.0648581698335
File No.11 | Filename: Agriculture_and_Residential_Transitional_zone.xml | Run time: 1.48279141535
File No.12 | Filename: agriculture_SantaCruz.xml | Run time: 0.0530900968644
F

File No.110 | Filename: Commercial_and_Residential_Transitional_LU_zone.xml | Run time: 1.44561977036
File No.111 | Filename: Commercial_LU_Zone.xml | Run time: 1.46769246221
File No.112 | Filename: commercial_site_Manchester_proposed.xml | Run time: 0.0505240525546
File No.113 | Filename: community_centres.xml | Run time: 0.0527114564501
File No.114 | Filename: Community_Names.xml | Run time: 0.0407162881152
File No.115 | Filename: coneys_Hutia_Gbrownii.xml | Run time: 0.048073555789
File No.116 | Filename: constituency_boundaries.xml | Run time: 0.0750276869692
File No.117 | Filename: Coral_Monitoring_Sites07.xml | Run time: 0.0429436892376
File No.118 | Filename: Coral_reef_map_2008_RC_JA.xml | Run time: 0.0423250654621
File No.119 | Filename: coral_sites.xml | Run time: 0.0507195945526
File No.120 | Filename: court_houses.xml | Run time: 0.0556988048829
File No.121 | Filename: craft_markets.xml | Run time: 0.0530709870782
File No.122 | Filename: CriticalFacilities_Cemetaries.xml | 

File No.216 | Filename: Geology_Stratadip.xml | Run time: 0.0547304275793
File No.217 | Filename: Geology_StrataDip_Point.xml | Run time: 0.0519008459858
File No.218 | Filename: Geology_SymbolGeoFormation.xml | Run time: 0.0571858128948
File No.219 | Filename: Goat_Island_Excavation_and_Reclaimation_Works.xml | Run time: 0.0404007544366
File No.220 | Filename: Green_Space_LU_Zone.xml | Run time: 1.46587569928
File No.221 | Filename: guano_caves.xml | Run time: 0.0477353570153
File No.222 | Filename: Gullies_drains_rivamp.xml | Run time: 0.0439440643227
File No.223 | Filename: habitat.xml | Run time: 0.0459825896515
File No.224 | Filename: HanoverExistingLanduse.xml | Run time: 0.0534207406064
File No.225 | Filename: HanoverProposals.xml | Run time: 0.130990029136
File No.226 | Filename: harmons.xml | Run time: 0.0353068853893
File No.227 | Filename: HazVul_Earthquake_Buffer_SC.xml | Run time: 0.0522923743954
File No.228 | Filename: HazVul_Flood_Harmons_2002.xml | Run time: 0.0472536126

File No.317 | Filename: ja_marine_overwintering_shorebird_areas_jan06.xml | Run time: 0.038523551256
File No.318 | Filename: Jamaica_caves.xml | Run time: 0.0406714023383
File No.319 | Filename: jm_terr_target_caves_JAD2001.xml | Run time: 0.0363459244604
File No.320 | Filename: Kingston.xml | Run time: 0.0302712345278
File No.321 | Filename: Kingston_Excavation_and_Reclaimation_Works.xml | Run time: 0.0345073852657
File No.322 | Filename: Kingston_Habour_Wetlands.xml | Run time: 0.0338278768228
File No.323 | Filename: KSA_Proposals_2017_04_19.xml | Run time: 0.5118147364
File No.324 | Filename: Land_Parcels_Annotto_Bay.xml | Run time: 0.0745423872834
File No.325 | Filename: land_use_Jamaica.xml | Run time: 0.057356912143
File No.326 | Filename: land_use_Manchester_existing.xml | Run time: 0.0525972421468
File No.327 | Filename: land_use_Portland_proposed.xml | Run time: 0.061937039032
File No.328 | Filename: Landslide.xml | Run time: 0.0408749437817
File No.329 | Filename: landslide_h

File No.424 | Filename: PC_50yr_2_9_3_6m.xml | Run time: 0.0723176526427
File No.425 | Filename: pedestrian_crossings_NWA.xml | Run time: 0.0573324693933
File No.426 | Filename: pharmacies.xml | Run time: 0.0529523286386
File No.427 | Filename: PIPELINE.xml | Run time: 0.0439911721676
File No.428 | Filename: pipelines_network_NIC.xml | Run time: 0.0570400452235
File No.429 | Filename: pipelines_network_NWC.xml | Run time: 0.0583146235194
File No.430 | Filename: police.xml | Run time: 0.0415477860202
File No.431 | Filename: pondbased.xml | Run time: 0.034520273261
File No.432 | Filename: Port_Antonion.xml | Run time: 0.0319760052192
File No.433 | Filename: Port_Maria.xml | Run time: 0.0356459729903
File No.434 | Filename: Portland_Bight_Historic_Sites.xml | Run time: 0.0430601256091
File No.435 | Filename: Portland_Bight_protected_Area_Ports.xml | Run time: 0.0403425362508
File No.436 | Filename: Portland_Bight_Ramsar_Wetland.xml | Run time: 0.0543109011109
File No.437 | Filename: Portl

File No.533 | Filename: storm_hurricane_tracks.xml | Run time: 0.0471669519802
File No.534 | Filename: storm_surge_HAllen_1980.xml | Run time: 0.044626683661
File No.535 | Filename: storm_surge_Montego_Bay_100yr.xml | Run time: 0.0503351767611
File No.536 | Filename: strata.xml | Run time: 0.0462932347801
File No.537 | Filename: Stream_gauge_stations.xml | Run time: 0.0428379187931
File No.538 | Filename: Sub_Watershed_management_units.xml | Run time: 0.054218907489
File No.539 | Filename: subdivisions_StElizabeth.xml | Run time: 0.0632667246183
File No.540 | Filename: SubRegional_Centre_Service_Area.xml | Run time: 0.0443115943962
File No.541 | Filename: substrates_ik.xml | Run time: 0.0342380706049
File No.542 | Filename: Surface_water_sources.xml | Run time: 0.0721341098129
File No.543 | Filename: SW_Goat_Island_Container_Hub_Bentic_Coverage.xml | Run time: 0.0323186481293
File No.544 | Filename: swamp.xml | Run time: 0.0375022887301
File No.545 | Filename: swift_river_chelsea.xml |

In [7]:
import csv

res = MissingFiles
csvfile = r"J:\David Jarvis\JAMNSP Jamaica NSP\ArcGIS\Metadata\metadata_import_process\5 Missing_xml_file_report\XML_Files_Not_Updated.csv"

#Assuming res is a flat list
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in res:
        writer.writerow([val])    